# Projet d'analyse d'alimentation des citoyens de pays différents

## Partie de nettoyage des données

Dans un premier et dans le but de faciliter la compréhension des données, l'objectif est d'analyser celles-ci quantitativement et qualitativement. 

La partie suivante inclue la mise en évidence des valeurs manquantes avec au moins 3 méthodes de traitement adatpées aux variables concernées et d'identification et quantification de valeurs abbérantes.

### Imports des modules

In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

On lit le fichier `fr.openfoodfacts.org.products.csv` avec le séparateur `\t`. Les données seront dans la variable `data`

In [90]:
data = pd.read_csv("fr.openfoodfacts.org.products.csv", sep="\t")

/tmp/ipykernel_39786/1111784718.py:1: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("fr.openfoodfacts.org.products.csv", sep="\t")


### Analyse quantitative des données

In [ ]:
# On lis les valeurs non definies
missing = data.isna()
# On les compte et on créer un nouveau DataFrame
missing_sum = pd.DataFrame(missing.sum(), columns=['nb_manquants'])
# On calcul et ajoute le ratio du nombre de valeurs non définies au DataFrame
missing_sum["ratio"] = ((missing_sum["nb_manquants"] / len(data)) )

# On va construire un graphique pour nous montrer le nombre de valeurs manquantes
plt.title("Ratio de valeurs manquantes par colonnes")
plt.ylabel("Ratio")
plt.xlabel("Colonnes contenant des valeurs manquantes")
plt.xticks([])
plt.plot(missing_sum.sort_values(["ratio"])["ratio"])
plt.show()
# On garde les ratio == 1
missing_sum_1 = missing_sum[missing_sum["ratio"] == 1]

ratio_missing = missing_sum["nb_manquants"].sum() / data.size

print(f"Sur tout le dataset, on a un taux de valeurs manquantes qui équivaut à {(ratio_missing*100):.2f} %.")
print(f"Sur {len(missing_sum)} colonnes, il y en a {len(missing_sum_1)} qui n'ont aucune valeurs")
print(f"Les colonnes qui n'ont aucune valeurs sont les suivantes :")
print(missing_sum_1.index.to_numpy())

La conclusion de cette analyse est qu'actuellement nous disposons de 25 % de données exploitables. On aura certainement une valeur différente après le traitement des valeurs manquantes.

## conservation des données utiles au projet uniquement

In [91]:

data = data.filter([
    "url",
    "product_name",
    "generic_name",
    "quantity", 
    "packaging",
    "packaging_tags",
    "countries_tags",
    "image_url",
    "image_small_url",
    "nutrition-score-fr_100g"
    ], axis=1)

missing_ratio = pd.DataFrame(data.isna().sum(), columns=['nb_manquants']).sum() / data.size

print(f"Après conservation des données utiles au projet, il n'y a plus que {(missing_ratio[0]*100):.2f} % de valeurs manquantes.")

Après conservation des données utiles au projet, il n'y a plus que 47.14 % de valeurs manquantes.


# Traitement des erreurs sur les variables générales

### Les champs `product_name` et `generic_name`

On pourra utiliser principalement le `product_name` pour récupérer le nom du produit et en cas d'erreur essayer d'utiliser le `generic_name`.
Ainsi on peut donc d'abord corriger les données de `generic_name` pour être sûr que la valeur qui sera possiblement remplacée dans `generic_name` sera valide. Ensuite on pourra corriger les autres données de `product_name` qui n'ont pas été remplacée mais qui malgré tout ne sont pas acceptables.

Si malgré tout il y a des valeurs qui ne sont pas définies, on gardera malgré tout ces lignes car ce qui compte est d'effectuer des comparaisons avec d'autres données. De plus, si on ne conservait pas ces lignes, comme on peut le voir dans le script, il ne nous resterait peu de données à exploiter.

Les traitements à effectuer sur `generic_name` sont les suivants :
- Ne pas prendre en compte les nulls
- Il y a des champs avec uniquement un espace à l'intérieur. On ne peut corriger ces lignes
- Ne pas prendre en compte les noms trop long car sinon ce n'est plus un nom mais une description. On ne peut corriger ces lignes

Les traitements à effectuer sur `product_name` sont les suivants:
- Ne pas prendre en compte les nulls
- Remplacer les valeurs non valide par sa valeur `generic_name`, `generic_name` étant valide

In [92]:
# -------------
# generic_name

# On conserve les non null
generic_exists = data["generic_name"].notna()
# On conserve les lignes qui ne contiennent pas uniquement des espaces
generic_no_space = data["generic_name"].ne(" ")
# On conserve les noms en dessous de 50 caractères
generic_not_long = data["generic_name"].str.len() < 50
# Test booléen
generic = generic_exists & generic_no_space & generic_not_long
# Table qui contient les éléments qui respectent les critères précédents
generic_match = data.loc[generic, ["generic_name"]]
print(f"Il y a {generic_match.size} generic_names sur {data.shape[0]} qui sont considérés comme valide, soit {((generic_match.size/data.shape[0])*100):.2f} %")

# -------------
# product_name

# Retirer les nulls
product_exists = data["product_name"].notna()
# Table qui contient les éléments qui respectent les critères précédents
product_match = data.loc[product_exists, ["product_name"]]

# Colonnes qui ne contiennent pas de product_name mais contiennent un generic_name
no_product_with_generic = (~product_exists) & generic
print(f"Il y a {no_product_with_generic.value_counts()[True]} lignes pour lesquelles il existe un generic_name valide mais pas de product_name valide")

# Remplacer les product_name non valide par son generic_name valide dans data
data.loc[no_product_with_generic, ["product_name"]] = data.loc[no_product_with_generic, ["generic_name"]]

# On peut désormait retirer generic_name qui n'est plus d'aucune utilité
data.drop(["generic_name"], axis=1)

Il y a 21991 generic_names sur 242222 qui sont considérés comme valide, soit 9.08 %
Il y a 31 lignes pour lesquelles il existe un generic_name valide mais pas de product_name valide


,url,product_name,quantity,packaging,packaging_tags,countries,countries_tags,image_url,image_small_url,nutrition-score-fr_100g
0,http://world-fr.openfoodfacts.org/produit/0000...,Farine de blé noir,1kg,NaN,NaN,en:FR,en:france,NaN,NaN,NaN
1,http://world-fr.openfoodfacts.org/produit/0000...,Banana Chips Sweetened (Whole),NaN,NaN,NaN,US,en:united-states,NaN,NaN,14.0
2,http://world-fr.openfoodfacts.org/produit/0000...,Peanuts,NaN,NaN,NaN,US,en:united-states,NaN,NaN,0.0
3,http://world-fr.openfoodfacts.org/produit/0000...,Organic Salted Nut Mix,NaN,NaN,NaN,US,en:united-states,NaN,NaN,12.0
4,http://world-fr.openfoodfacts.org/produit/0000...,Organic Polenta,NaN,NaN,NaN,US,en:united-states,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
242217,http://world-fr.openfoodfacts.org/produit/3564...,Quenelles de volaille au naturel,400 g,Conserve,conserve,France,en:france,http://fr.openfoodfacts.org/images/products/35...,http://fr.openfoodfacts.org/images/products/35...,9.0
242218,http://world-fr.openfoodfacts.org/produit/3564...,Pirates Multi-Céréales Goût Chocolat,300 g,"Plastique,Papier,Carton","plastique,papier,carton",France,en:france,http://fr.openfoodfacts.org/images/products/35...,http://fr.openfoodfacts.org/images/products/35...,16.0
242219,http://world-fr.openfoodfacts.org/produit/3564...,Chèvre à dorer,100 g,"carton,sous-vide","carton,sous-vide",France,en:france,http://fr.openfoodfacts.org/images/products/35...,http://fr.openfoodfacts.org/images/products/35...,12.0
242220,http://world-fr.openfoodfacts.org/produit/3564...,Céréales Fourrées Chocolat,750 g (2 x 375 g),"Boîte,Carton,Sachet,Plastique","boite,carton,sachet,plastique",France,en:france,http://fr.openfoodfacts.org/images/products/35...,http://fr.openfoodfacts.org/images/products/35...,13.0


### La colonne `countries_tags`

Pour le projet il est important de recenser les pays qui sont mentionnés dans nos données. Dans les données proposée, il y avais également le champ `countries` mais j'ai remarqué qu'il y avait quelques erreurs dedans. Je préfère créer un dictionnaire qui va me permettre de convertir les tags en nom pour pouvoir les utiliser plus tard.

In [101]:
test = data["countries_tags"].value_counts()

pays_acceptes = {
    "en:united-states": "Etats-Unis", 
    "en:france": "France", 
    "en:united-kingdom": "Royaume-Uni",
    "en:germany": "Allemagne",
    "en:switzerland": "Suisse",
    "en:spain": "Espagne",
    "en:canada": "Canada",
    "en:belgium": "Belgique",
    "en:australia": "Australie",
    "en:portugal": "Portugal",
    "en:russia": "Russie",
    "en:italy": "Italie",
    "en:reunion": "France, Réunion",
    "en:austria": "Autriche",
    "en:guadeloupe": "France, Guadeloupe",
    "en:netherlands": "Pays-Bas",
    "en:ireland": "Irlande",
    "en:martinique": "France, Martinique",
    "en:luxembourg": "Luxembourg",
    "en:french-guiana": "Guyane Française",
    "en:new-caledonia": "Nouvelle Calédonie",
    "en:czech-republic": "République Tchèque",
    "en:french-polynesia": "Polynésie Française",
    "en:morocco": "Maroc",
    "en:taiwan": "Taiwan",
    "en:tunisia": "Tunisie"
}

# Selection des données qui seront intéressantes pour le projet

Cela permet surtout de ne pas trop utiliser la mémoire pour les données qui ne me seront pas utiles. On gardera les `tags` pour faire nos analyses.
On peut également dors et déjà éliminer les 17 colonnes qui n'ont aucune valeur. Il n'y a donc pas les éléments suivant dans la liste :

`no_nutriments` ; `ingredients_from_palm_oil` ; `ingredients_that_may_be_from_palm_oil` ; `nutrition_grade_uk` ; `butyric-acid_100g` ; `caproic-acid_100g` ; `lignoceric-acid_100g` ; `cerotic-acid_100g` ; `montanic-acid_100g` ; `melissic-acid_100g` ; `elaidic-acid_100g` ; `mead-acid_100g` ; `erucic-acid_100g` ; `nervonic-acid_100g` ; `chlorophyl_100g` ; `glycemic-index_100g` ; `water-hardness_100g`